In [1]:
import numpy as np
import pandas as pd
import lxml.etree as etree
import sqlite3 as sql
from preprocess import DB
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt

In [2]:
db = DB(db_route = '../data/database.sqlite')

In [3]:
db.combineCardFoulDF()

  0%|          | 0/14217 [00:00<?, ?it/s]

IntCastingNaNError: Cannot convert non-finite values (NA or inf) to integer

In [11]:
xml = db.match_team['foulcommit'].iloc[3]
foul_df = pd.read_xml(xml)
### investigagte what goes on here!

,stats,event_incident_typefk,elapsed,player2,player1,sortorder,team,n,type,id,subtype,elapsed_plus
0,NaN,37,1,23579.0,30843.0,1,10260,259,foulcommit,2525273,None,NaN
1,NaN,37,7,43248.0,35411.0,0,10261,265,foulcommit,2525338,None,NaN
2,NaN,37,15,150466.0,35327.0,2,10260,267,foulcommit,2525440,None,NaN
3,NaN,37,17,35411.0,30373.0,0,10260,269,foulcommit,2525468,None,NaN
4,NaN,37,20,70409.0,24899.0,0,10261,272,foulcommit,2525499,None,NaN
5,NaN,37,24,35327.0,150466.0,1,10261,278,foulcommit,2525575,None,NaN
6,NaN,43,33,NaN,24899.0,0,10261,286,foulcommit,2525684,None,NaN
7,NaN,37,33,24899.0,24154.0,3,10260,289,foulcommit,2525695,None,NaN
8,NaN,3,36,NaN,43248.0,0,10260,291,foulcommit,2525725,hands,NaN
9,NaN,37,38,41191.0,70409.0,2,10260,296,foulcommit,2525760,None,NaN


In [8]:
#base_elapsed = pd.DataFrame({'elapsed_foul' : list(range(120))})
#df1 = pd.merge(base_elapsed, db.unravelFoulDF(3), how = 'left', on = 'elapsed_foul')
#db.unravelFoulDF(3)

In [15]:
for key in list(df1.dtypes.keys()):
    if df1.dtypes[key] == np.dtype('float64'):
        if key == 'match_id':
            mid = df1[~df1['match_id'].isna()]['match_id'].iloc[0]
            df1 = df1.fillna({key : mid}).astype({str(key) : int})
        df1 = df1.fillna({key : 0}).astype({str(key) : int})

In [19]:
df1.columns

Index(['elapsed_foul', 'card_type', 'event_incident_typefk_foul', 'foul_id',
       'injury_time', 'player1_id_foul', 'player2_id_foul', 'subtype', 'team',
       'type', 'match_id', 'player1_name_foul', 'player2_name_foul'],
      dtype='object')

In [20]:
join = pd.merge_asof(
    df1, db.unravelCardDF(0),
    left_on = 'elapsed_foul',
    right_on = 'elapsed_card',
    left_by = 'player1_id_foul',
    right_by = 'player1_id_card',
    direction = 'nearest',
    tolerance = 2
)

In [21]:
join

,elapsed_foul,card_type_x,event_incident_typefk_foul,foul_id,injury_time,player1_id_foul,player2_id_foul,subtype,team,type,...,match_id_y,card_color,elapsed_card,event_incident_typefk_card,goal_type,card_id,player1_id_card,card_subtype,team_card,card_type_y
0,0,NaN,0,0,NaN,0,0,NaN,0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,NaN,37,378824,NaN,25518,32569,NaN,10261,foulcommit,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,NaN,37,378826,NaN,30929,24157,NaN,10261,foulcommit,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,NaN,37,378830,NaN,29581,24148,NaN,10261,foulcommit,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,NaN,0,0,NaN,0,0,NaN,0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
116,115,NaN,0,0,NaN,0,0,NaN,0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
117,116,NaN,0,0,NaN,0,0,NaN,0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
118,117,NaN,0,0,NaN,0,0,NaN,0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
119,118,NaN,0,0,NaN,0,0,NaN,0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
